<a href="https://colab.research.google.com/github/ManuelBagasina/solarspell/blob/playground/RwandaB2RClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

Amy's notes: https://docs.google.com/document/d/1-4Lzom5cNE2BzIqJ8_i6_on2eXxcBIQJ5eD72E_QZZk/edit?tab=t.0#heading=h.3e1ixdtymtso

Kadeeja's: https://docs.google.com/document/d/178cewOnabbWzSV6WFniU7miAfDY8Ms2lro4aJ-tbO5c/edit?tab=t.0


# Setup

In [1]:
from google.colab import files
uploaded = files.upload()

Saving RwandaData_Cleaned1.xlsx to RwandaData_Cleaned1.xlsx


In [2]:
import pandas as pd

filename = 'RwandaData_Cleaned1.xlsx'
df = pd.read_excel(filename)

print(df.head())

         Date First name     Last name  Gender     Age range Occupation  \
0  3/28/2025      Eduard      Turasaba    Male  51 and above     Farmer   
1  3/22/2025   Thacienne    Nyirabenda  Female  51 and above     Farmer   
2  4/16/2025   Spelacie   Mukanyangezi  Female  51 and above     Farmer   
3  3/26/2025     Maurice     Kubwimana    Male         41-45     Farmer   
4  3/24/2025         JMV      Uwayezu     Male  51 and above     Farmer   

  Permanent residential address   District   Sector       Cell  ...  \
0              Eastern Province    Kirehe   Musaza     Kabuga   ...   
1             Southern Province    Muhanga  Kiyumba     Ruhina  ...   
2             Southern Province  Nyamagabe     Tare  Gasarenda  ...   
3             Southern Province    Kamonyi   Rukoma       Taba  ...   
4             Southern Province  Gisagara   Kigembe  Akabacuzi  ...   

  Num_Trained_Planning_2025B Planted_Other_Crops_2025A Area_Other_Crops_CA  \
0                          2                

# Cleaning

## Change Column Names

In [ ]:
# Mapping of new column names
#rename_map = {
    # Basic respondent info
    'Timestamp': 'Timestamp',
    'First name: (e.g. Jean Marie or Donatille)': 'First_Name',
    'Last name (e.g. Mugabo or Mukeshimana)': 'Last_Name',
    '1. Gender': 'Gender',
    '4. Age range': 'Age_Range',
    'Occupation/Akazi mukora?': 'Occupation',
    '5. Permanent residential address/ Aho ubarizwa cyangwa utuye ': 'Address',
    '6. District/ Akarere mutuyemo': 'District',
    '7. Sector/ Umurenge mutuyemo': 'Sector',
    '8. Cell/ Akagari mutuyemo': 'Cell',
    '9. Village/ Umudugudu mutuyemo': 'Village',
    '10. Farm Location/ Coordinates': 'Farm_Coords',
    '11. UPI of the Farm if Possible ': 'UPI',
    # Key identifiers
    'Have you ever used any content (videos, pdfs, etc) from the SolarSPELL digital library? (see picture below)': 'Used_SS',
    'What content did you access from the SolarSPELL library? Can you give a few examples? (Nayahe masomo warebye/ wize ukoresheje SolarSpell Library? Waduha ingero zayo masomo?)': 'SS_Content_Examples',
    'Do you agree or disagree with the following statement: The content from the SolarSPELL library was helpful to me in learning more about conservation agriculture. ': 'SS_Helpful_Learning_CA',
    'Do you agree or disagree with the following statement: I applied/used information I learned from the SolarSPELL library on my farm.': 'SS_Applied_on_Farm',
    'How have you used the information you learned from the SolarSPELL library content? ': 'SS_Usage_Examples',
    # Planting
    "1. A. Have you planted a CA demo plot of Maize? Mwashyizeho umurima w'ibigori wo kugerageza muburyo musigasira ubutaka?": 'Planted_CA_Maize',
    "B.  Have you planted an SP demo plot of Maize? Mwashyizeho umurima w'ibigori wo kugerageza uhinze muburyo busanzwe?": 'Planted_SP_Maize',
    # Numeric areas & yields
    '2. A. What was the area (are) you planted Maize using CA? Ibigori wabihinze ku buhe buso (Ari) Muburyo bubungabunga ubutaka? (Umubare gusa ntamagambo ajyamo)': 'Area_CA_Maize',
    'B. What was the area (Are) you planted Maize using SP? Ibi wabihinze ku buhe buso (Ari) muburyo busanzwe? (Umubare gusa ntamagambo ajyamo)': 'Area_SP_Maize',
    '3. A. What was the Yield of Maize (Kgs) in CA Demo plot? Ibigori mwasaruye ibiro bingahe mumurima wo kugerageza Muburyo bubungabunga ubutaka? (Umubare gusa ntamagambo ajyamo)': 'Yield_CA_Maize',
    'B. What was the Yield of Maize (Kgs) in SP Demo plot? Ibigori mwasaruye ibiro bingahe mumurima wo kugerageza Muburyo busanzwe? (Umubare gusa ntamagambo ajyamo)': 'Yield_SP_Maize',
    # Expansion
    'Did you Apply CA on an other plots? Ese hari ubundi butaka wakoreyeho ubuhinzi busigasira ubutaka?': 'Applied_CA_Expansion',
    "what was the area of that plot? ubuso bw'ubwo butaka (Ari) bungana gute? (Umubare gusa ntamagambo ajyamo)": 'Area_CA_Expansion',
    'What was the Yield harvested on that plot? ese havuyeho umusaruro ungana gute? (Umubare gusa ntamagambo ajyamo)': 'Yield_CA_Expansion',
    # Training
    'Have you trained/ shared with other people back in your community about conservation agriculture (CA). Hari abantu mwahuguye cg mwasangije ubumenyi kubijyanye n\'ubuhinzi bubungabunga ubutaka?': 'Trained_CA',
    '2. How many people did you train/ shared with back in your community about CA? Abantu mwahuguye cg mwasangije ubumenyi kubuhinzi busigasira ubutaka ni bangahe?': 'Num_Trained_Total',
    '3. How many people did you train/ shared with in your community who planted their crops under conservation agriculture in season 2025B? Mu bantu mwahuguye cg mwasangije ubumenyi, nibangahe bakoze ubuhinzi busigasira ubutaka muri sizoni 2025B?': 'Num_Trained_Planted_2025B',
    '3. How many people did you train in your community who are planning to plant crops under conservation agriculture in season 2025B? Mu bantu mwahuguye, nibangahe biteguye gukora ubuhinzi busigasira ubutaka muri sizoni 2025B?': 'Num_Trained_Planning_2025B',
    # Other crops
    'Did you plant other crops in season 2025A? Hari ibindi bihingwa waba warahinze muri sizoni 2025A?': 'Planted_Other_Crops_2025A',
    '6. What was the area you planted other crops using CA? Ibindi bihingwa bitari ibigori wabihinze ku buhe buso? (Umubare gusa ntamagambo ajyamo)': 'Area_Other_Crops_CA',
    'B. What was the area you planted other crops using SP? Ibindi bihingwa bitari ibigori wabihinze ku buhe buso? (Umubare gusa ntamagambo ajyamo)': 'Area_Other_Crops_SP',
    '5. What was the yield of crops planted using CA other than Maize in Season 2025A. Ni uwuhe musaruro bw\' ibindi bihingwa wahinze mu buryo busigasira ubutaka bitari Ibigori? (Umubare gusa ntamagambo ajyamo)': 'Yield_Other_CA_2025A',
    'B. What was the yield of crops planted using SP other than Maize in seaon 2025A. Ni uwuhe musaruro w\' ibindi bihingwa wahinze mu buryo busanzwe bitari Ibigori? (Umubare gusa ntamagambo ajyamo)': 'Yield_Other_SP_2025A',
    'What area are you planting on those crops': 'Area_Other_Crops_Total',
    'How many seasons have you practiced CA in your fields? Umaze guhinga muburyo busigasira ubutaka incuro zingahe?': 'Num_Seasons_CA',
    'Have you observed any benefits of using CA in your fields? Ese hari ibyiza byogukoresha ubuhinzi busigasira ubutaka wabonye?': 'CA_Benefits_Observed',
    'Are you planning to continue applying CA in your field? Wumva uzakomeza gukoresha ubuhinzi busigasira ubutaka?': 'CA_Continue_Plan'
#}

In [ ]:
# Change column Names
#effective_map = {orig: new for orig, new in rename_map.items() if orig in df.columns}
#df = df.rename(columns=effective_map)
#effective_map = {orig: new for orig, new in rename_map.items() if orig in df.columns}
#df = df.rename(columns=effective_map)

In [ ]:
# Checking New Columns
df.columns

Index(['Timestamp', 'First_Name', 'Last_Name', 'Gender', 'Age_Range',
       'Occupation', 'Address', 'District', 'Sector', 'Cell', 'Village',
       'Farm_Coords', 'UPI', 'Used_SS', 'SS_Content_Examples',
       'SS_Helpful_Learning_CA', 'SS_Applied_on_Farm', 'SS_Usage_Examples',
       'Planted_CA_Maize', 'Planted_SP_Maize', 'Area_CA_Maize',
       'Area_SP_Maize', 'Yield_CA_Maize', 'Yield_SP_Maize',
       'Applied_CA_Expansion', 'Area_CA_Expansion', 'Yield_CA_Expansion',
       'Trained_CA', 'Num_Trained_Total', 'Num_Trained_Planted_2025B',
       'Num_Trained_Planning_2025B', 'Planted_Other_Crops_2025A',
       'Area_Other_Crops_CA', 'Area_Other_Crops_SP', 'Yield_Other_CA_2025A',
       'Yield_Other_SP_2025A', 'Area_Other_Crops_Total', 'Num_Seasons_CA',
       'CA_Benefits_Observed', 'CA_Continue_Plan'],
      dtype='object')

## Translation

In [ ]:
# Translation

## Standardizing Nulls

In [3]:
# Standardize "No Data" Entries
df = df.replace({'N/A': pd.NA, 'N/Q': pd.NA})

## Data Type Conversions

In [4]:
# Create a list of your numeric columns
numeric_cols = [col for col in df.columns if col.startswith(('Area_', 'Yield_', 'Num_'))]

# Loop through each column and find rows containing letters
for col in numeric_cols:
    # Use .str.contains() to find any cell with letters (a-z, A-Z)
    # na=False ensures that actual Null values (NaN) don't cause an error
    problem_rows = df[df[col].astype(str).str.contains('[a-zA-Z]', na=False)]

    if not problem_rows.empty:
        print(f"--- Found text in column: {col} ---")
        print(problem_rows[[col]]) # Show the problematic values in that column

--- Found text in column: Area_CA_Maize ---
     Area_CA_Maize
219            NA 
226            NA 
228            NA 
342           8are
343            O.5
...            ...
3559           NaN
3560           NaN
3561           NaN
3565           NaN
3570           NaN

[500 rows x 1 columns]
--- Found text in column: Area_SP_Maize ---
     Area_SP_Maize
0              NaN
2              NaN
3              NaN
4              NaN
5              NaN
...            ...
3563           NaN
3565           NaN
3567          O.75
3569           NaN
3570           NaN

[2434 rows x 1 columns]
--- Found text in column: Yield_CA_Maize ---
     Yield_CA_Maize
0               NaN
2               NaN
5               NaN
6               NaN
7               NaN
...             ...
3560            NaN
3561            NaN
3563            NaN
3565            NaN
3570            NaN

[1714 rows x 1 columns]
--- Found text in column: Yield_SP_Maize ---
     Yield_SP_Maize
0               NaN
2           

In [5]:
import numpy as np

# --- 1. Define the Comprehensive Cleaning Function ---

def clean_and_convert(series):
    # Convert the entire series to a string
    s = series.astype(str)

    # Standardize all null-like text to a single type (np.nan)
    # We use .lower() to catch 'na', 'NA', etc., and strip() to remove spaces
    null_values = ['na', 'n/a', 'm/a', 'nan', 'n /a']
    s = s.str.lower().str.strip() # Make it lowercase and remove leading/trailing spaces
    s = s.replace(null_values, np.nan)

    # Fix typos: Replace the letter 'O' with the number '0'
    s = s.str.replace('o', '0', regex=False)

    # Remove units: Strip 'are' and 'kg' and any surrounding space
    s = s.str.replace(r'\s*are\s*', '', regex=True)
    s = s.str.replace(r'\s*kg\s*', '', regex=True)

    # Convert the cleaned series to a numeric type.
    # 'coerce' will turn any remaining problems into a proper Null (NaN).
    return pd.to_numeric(s, errors='coerce')


# --- 2. Identify the Columns to Clean ---

numeric_cols = [col for col in df.columns if col.startswith(('Area_', 'Yield_', 'Num_'))]

# --- 3. Apply the Function to Each Column ---

print("--- Data types BEFORE cleaning ---")
print(df[numeric_cols].dtypes)

for col in numeric_cols:
    df[col] = clean_and_convert(df[col])

print("\n--- Data types AFTER cleaning ---")
print(df[numeric_cols].dtypes)

# --- 4. (Optional) Verify one of the columns ---
print("\n--- Example of cleaned column: Area_CA_Maize ---")
print(df['Area_CA_Maize'].head(10))

--- Data types BEFORE cleaning ---
Area_CA_Maize                 object
Area_SP_Maize                 object
Yield_CA_Maize                object
Yield_SP_Maize                object
Area_CA_Expansion             object
Yield_CA_Expansion            object
Num_Trained_Total             object
Num_Trained_Planted_2025B     object
Num_Trained_Planning_2025B    object
Area_Other_Crops_CA           object
Area_Other_Crops_SP           object
Yield_Other_CA_2025A          object
Yield_Other_SP_2025A          object
Area_Other_Crops_Total        object
Num_Seasons_CA                 int64
dtype: object

--- Data types AFTER cleaning ---
Area_CA_Maize                 float64
Area_SP_Maize                 float64
Yield_CA_Maize                float64
Yield_SP_Maize                float64
Area_CA_Expansion             float64
Yield_CA_Expansion            float64
Num_Trained_Total             float64
Num_Trained_Planted_2025B     float64
Num_Trained_Planning_2025B    float64
Area_Other_Crops_CA

## Create Core Analysis Variables

In [ ]:
# Create the Core Analysis Variable

# Analysis

# Save

In [ ]:
# Save cleaned dataset
df.to_csv('solarSPELL_survey_cleaned.csv', index=False)
print("Cleaned dataset saved to solarSPELL_survey_cleaned.csv")


Cleaned dataset saved to solarSPELL_survey_cleaned.csv
